# Q7: Producer–Consumer Problem in Python using Semaphores

This notebook demonstrates how to handle the classical **Producer–Consumer** problem in Python using **semaphores** to prevent race conditions and ensure **mutual exclusion**.

We use:
- A **shared buffer** (list) as the critical resource.
- Two semaphores:
  - `empty_slots` – counts how many free positions are left in the buffer.
  - `full_slots` – counts how many items are available to consume.
- A **mutex lock** (`buffer_lock`) to provide **mutual exclusion** when the buffer is accessed.


In [ ]:
import threading
import time
import random

NUM_ITEMS = 10
BUFFER_SIZE = 5                  
buffer = []                      
empty_slots = threading.Semaphore(BUFFER_SIZE)  
full_slots = threading.Semaphore(0)             
buffer_lock = threading.Lock()

def producer(producer_id):
    """Produce NUM_ITEMS items and then stop."""
    for item in range(1, NUM_ITEMS + 1):
        time.sleep(random.uniform(0.2, 0.7))


        empty_slots.acquire()

        with buffer_lock:
            buffer.append(item)
            print(f"Producer {producer_id} produced {item}. Buffer: {buffer}")
      
        full_slots.release()
def consumer(consumer_id):
    """Consume NUM_ITEMS items and then stop."""
    for _ in range(NUM_ITEMS):      
        full_slots.acquire()
     
        with buffer_lock:
            item = buffer.pop(0)
            print(f"Consumer {consumer_id} consumed {item}. Buffer: {buffer}")
      
        empty_slots.release()
        time.sleep(random.uniform(0.2, 0.7))  
if __name__ == "__main__":
   
    producer_thread = threading.Thread(target=producer, args=(1,))
    consumer_thread = threading.Thread(target=consumer, args=(1,))
    producer_thread.start()
    consumer_thread.start()
    producer_thread.join()
    consumer_thread.join()
    print("\nDemo finished (all items produced and consumed).")
producer_thread = threading.Thread(target=producer, args=(1,))
consumer_thread = threading.Thread(target=consumer, args=(1,))
producer_thread.start()
consumer_thread.start()
producer_thread.join()
consumer_thread.join()
print("\nDemo finished (all items produced and consumed).")


## Explanation: How Semaphores Prevent Race Conditions

1. **Critical Section & Mutual Exclusion**
   - The shared list `buffer` is the critical resource.
   - We protect it with `buffer_lock` (a mutex). Only one thread at a time can    execute inside the `with buffer_lock:` block.

2. **Semaphores for Synchronization**
   - `empty_slots` is initialized to `BUFFER_SIZE`. A producer must call      `empty_slots.acquire()` before inserting. If the buffer is full, it will block,      preventing overflow.
   - `full_slots` is initialized to `0`. A consumer must call `full_slots.acquire()` before      removing an item. If the buffer is empty, it will block, preventing underflow.

3. **No Race Conditions**
   - All modifications to `buffer` (append/pop) happen under `buffer_lock`, ensuring mutual      exclusion.
   - Semaphores coordinate the order (producer must produce before consumer consumes),      so the buffer is never read or written in an inconsistent state.

This satisfies the requirements of the classical Producer–Consumer problem: **mutual exclusion** and **correct synchronization** using semaphores.
